# Databrick Tutorial
### Creating a databricks workspace and connecting to azure storage
1. Azure: Go to the azure and search for databricks. Enter the details and create a databricks workspace
2. Azure: search for "Access Connector for databrick" in azure and create one
3. Azure: Create a storage account and two containers one for source and one destination
4. Azure: Go to storage account and give "storage blob data contributor" permission to the "access connector for databricks"
5. Databricks: Go to catalogs and create a new storage credential and provide "access connector ID" to the storage credentials
6. Databricks: create a new external location and provde the file path in the following format "abfss://<container_name>@<storage_account_name>.dfs.core.windows.net/<path>/file.csv. also select the storage credentials you created earlier
7. After the access the file from the notebook using the below command
    file_path = "abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv"
    file_content = dbutils.fs.head(file_path)
    print(file_content)


**Accessing a file in the Azure ADLS 2**

In [0]:
#Option 1
file_path = "abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv"
file_content = dbutils.fs.head(file_path)
print(file_content) 

#Option 2
# df = spark.read.option("header", "true").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
# df.display()

**Listing files in a contaioner**

In [0]:
display(dbutils.fs.ls("abfss://source@rxdbstorageaccount.dfs.core.windows.net/"))



**Copy file from one container to another**


In [0]:
df = spark.read.option("header", "true").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
df.write.mode("overwrite").csv("abfss://destination@rxdbstorageaccount.dfs.core.windows.net/Sales_Copy")


In [0]:
dbutils.fs.ls('/databricks-datasets/')

## Transformations

**Select() and Drop(**

In [0]:
# Read the file from the Azure storage into a spark data frame
df = spark.read.option("header", "true").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
#display the data in spark environment
#df.display()
df_select = df.select("Item_Identifier", "Item_Outlet_Sales", "Item_Weight")

#df_select.display()
# not formatted highly and is suppoprted in opyspark
#df_select.show()

# drop specific columns from the data frame
df_drop=df_select.drop("Item_Weight")
df_drop.display()



**withColumnRenamed(), withColumn(), lit(), cast(), when()**

In [0]:
# df = spark.read.option("header", "True").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
# df = df.withColumnRenamed("Item_Identifier", "ItemID")
# df.display()

import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, lit
df = spark.read.option("header", "True").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
from pyspark.sql.types import FloatType
df = df.withColumn("Item_Weight_Category", when(col("Item_Weight").cast(FloatType())>10, lit("high")).otherwise(when (col("Item_Weight").cast(FloatType())>5, lit("medium")).otherwise(lit("low"))))

df.display()



**alias(), cast(), upper()**

In [0]:
from pyspark.sql.functions import col, upper
df=spark.read.option("header", "true").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
#df.display()
df=df.select("Item_Fat_Content", "Item_Identifier", col("Item_Type").alias("Item"), "Item_Weight", "Item_Visibility", "Item_MRP", col("Item_Outlet_Sales").cast("float"), "Outlet_Establishment_Year", "Outlet_Size", upper(col("Outlet_Location_Type")), "Outlet_Type")
df.display()


**schema()**

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

schema= StructType([
    StructField("Item_Identifier", StringType(), True),
    StructField("Item_Weight", FloatType(), True),
    StructField("Item_Fat_Content", StringType(), True),
    StructField("Item_Visibility", FloatType(), True),
    StructField("Item_Type", StringType(), True),
    StructField("Item_MRP", FloatType(), True),
    StructField("Outlet_Identifier", StringType(), True),
    StructField("Outlet_Establishment_Year", IntegerType(), True),
    StructField("Outlet_Size", StringType(), True),
    StructField("Outlet_Location_Type", StringType(), True),
    StructField("Outlet_Type", StringType(), True),
    StructField("Item_Outlet_Sales", FloatType(), True)
])

df = spark.read.option("header", "true").schema(schema).csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")

# df.schema
# df.printSchema()
# df.schema.fieldNames()
# df.columns


**String Transformations - lower()/upper(), trim()/ltrim(), rtrim(), substring(), concat()/concat_ws()**

In [0]:

from pyspark.sql.functions import lower, upper, col, concat, trim, substring
df=spark.read.option("header", "true").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
df= df.select(lower("Item_Identifier").alias("Item_Identifier"), "Item_Weight", upper("Item_Fat_Content"), "Item_Visibility", trim(col("Item_Type")), "Item_MRP", "Outlet_Identifier", "Outlet_Establishment_Year", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type", "Item_Outlet_Sales", concat("Outlet_Identifier", lit("_"),"Outlet_Establishment_Year").alias("Outlet"), substring("Outlet_Establishment_Year",-2,2)).alias("Year")
df.display()

**String Transformation - regexp_extract()/regexp_repplace()**

In [0]:
from pyspark.sql.functions import regexp_extract, regexp_replace
df=spark.read.option("header", "true").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
#df.display()
df= df.select("Item_Identifier", "Item_Weight", regexp_replace("Item_Fat_Content","\s*Low\s+Fat\s*","High Fat"), "Item_Visibility", "Item_Type", "Item_MRP", "Outlet_Identifier", "Outlet_Establishment_Year", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type", "Item_Outlet_Sales", regexp_extract("Item_Fat_Content", "^(Low|Regular)", 1).alias("Item_Fat_Content",))
df.display()


**Numeric Transformations round()/bround(),floor(), ceil()**

In [0]:
from pyspark.sql.functions import round, bround, floor, ceil
df = spark.read.option("header", "True").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
df = df.select("Item_Identifier", round("Item_Weight",3), "Item_Fat_Content", bround("Item_Visibility",3), "Item_Visibility", "Item_Type", floor("Item_MRP"), "Outlet_Identifier", "Outlet_Establishment_Year", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type", "Item_Outlet_Sales")
df.display()

**Null Handling - fillna, filter, isin**

In [0]:
from pyspark.sql.types import StringType, StructField, StructType, FloatType, IntegerType
from pyspark.sql.functions import lit, col, when

schema = StructType([
    StructField('Item_Identifier', StringType(), True),
    StructField('Item_Weight', FloatType(), True),
    StructField('Item_Fat_Content', StringType(), True),
    StructField('Item_Visibility', FloatType(), True),
    StructField('Item_Type', StringType(), True),
    StructField('Item_MRP', FloatType(), True),
    StructField('Outlet_Identifier', StringType(), True),
    StructField('Outlet_Establishment_Year', IntegerType(), True),
    StructField('Outlet_Size', StringType(), True),
    StructField('Outlet_Location_Type', StringType(), True),
    StructField('Outlet_Type', StringType(), True),
    StructField('Item_Outlet_Sales', FloatType(), True  )
])
df = spark.read.option("header", "True").schema(schema).csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
# df.display()
df=df.fillna({"Item_Identifier": "Unknown", "Item_Weight": 0.0, "Item_Fat_Content":"Unknown", "Item_Visibility": 0.0, "Item_Type": "Unknown", "Item_MRP": 0.0, "Outlet_Identifier": "Unknown", "Outlet_Establishment_Year": 0, "Outlet_Size": "Unknown", "Outlet_Location_Type": "Unknown", "Outlet_Type": "Unknown", "Item_Outlet_Sales": 0.0})
#df_fill.display()
#df=df.filter((col("Outlet_Size")=='Unknown') | ((col("Outlet_Location_Type")=='Unknown') | (col("Outlet_Type")=='Unknown')))
df=df.filter((col("Item_Type").isin(["Fruits and Vegetables", "Snack Foods"])))
df.display()


**Null Handling - fillna, filter, na.replace**

In [0]:
from pyspark.sql.types import StringType, StructField, StructType, FloatType, IntegerType
from pyspark.sql.functions import lit, col, when
df = spark.read.option("header", "True").schema(schema).csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
# df=df.dropna(how='all')
# df=df.dropna(how='any')
# df=df.dropna(thresh=1)
# df=df.dropna(subset=["Outlet_Size", "Outlet_Location_Type", "Outlet_Type"])
#df=df.filter(col("Item_Weight").isNull())
#df=df.filter(col("Item_Weight").isNotNull())
df=df.na.replace(["", "Null", "n/a"], None)
df.display()


**Row Level Operations - filter, where**

In [0]:
from pyspark.sql.types import StringType, StructField, StructType, FloatType, IntegerType
from pyspark.sql.functions import lit, col, when
df = spark.read.option("header", "True").schema(schema).csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
df = df.where((col("Item_Weight") > 10) & (col("Item_Type").isin(["Fruits and Vegetables", "Snack Foods"])))
df = df.display()


**distinct(),dropDuplicates(), limit()**


In [0]:
df = spark.read.option("header", "True").schema(schema).csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
#df.display()
df=df.distinct()
df=df.dropDuplicates(["Item_Fat_Content"])
df.limit(3).display()

**sample() orderBy() / sort()**

In [0]:
df = spark.read.option("header", "True").schema(schema).csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
df=df.sample(withReplacement=False,fraction=0.1)
#df.display()
#df=df.orderBy("Item_Type",ascending = False)
df=df.sort(["Item_Type", "Outlet_Identifier"], ascending=[True, True])
df.display()

### Aggregations


**groupBy(),agg()**

In [0]:
from pyspark.sql.functions import sum,avg,min,max,count

df = spark.read.option("header", "True").schema(schema).csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
#df=df.groupBy("Item_Fat_Content").agg(count("Item_Outlet_Sales"))
# df=df.groupBy("Item_Fat_Content").agg(sum(col("Item_Outlet_Sales")).alias("Total Sales"))
#df=df.groupBy("Item_Fat_Content").agg(avg(col("Item_Outlet_Sales")).alias("Average Sales"))
#df=df.groupBy("Item_Fat_Content").agg(min(col("Item_Outlet_Sales")).alias("Lowest Sales"))
df=df.groupBy("Item_Fat_Content").agg(max(col("Item_Outlet_Sales")).alias("Highest Sales"))
df.display()

**pivot(),  unpivot(), cube() / rollup()**

In [0]:
from pyspark.sql.functions import col
df= spark.read.option("header", "True").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
df=df.select("Item_Fat_Content","Item_type", col("Item_Outlet_Sales").cast("float"))
#df.display()
# df=df.groupBy("Item_Fat_Content").pivot("Item_type").sum("Item_Outlet_Sales")
# df.display()
df=df.groupby("Item_type").pivot("Item_Fat_Content").avg("Item_Outlet_Sales")
#df.display()
df=df.selectExpr("Item_type", 
                 "stack(3, 'LF', `LF`, 'Regular', `Regular`, 'reg', `reg`) as (Item_Fat_Content, `Average Sales`)"
)
df.display()

**cube() / rollup()**

In [0]:
from pyspark.sql.functions import col, sum
df= spark.read.option("header", "True").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
df=df.select("Item_Fat_Content","Item_type", col("Item_Outlet_Sales").cast("float"))
#df.display()
# df=df.rollup("Item_Fat_Content", "Item_type").agg(sum(col("Item_Outlet_Sales")).alias("Total Sales"))
# df.filter(col("Item_type").isNull()).display()
df = df.cube("Item_Fat_Content", "Item_type").agg(sum(col("Item_Outlet_Sales")).alias("Total Sales"))
# df.display()
#df=df.filter(col("Item_type").isNull() & col("Item_Fat_Content").isNull())
# df.display()
# df=df.filter(col("Item_type").isNull())
# df.display()
df=df.filter(col("Item_Fat_Content").isNull())
df.display()


### Window Functions

**row_number(), rank()/dense_rank(), lag() / lead(), ntile(), sum() over, avg() over, etc.**

In [0]:
from pyspark.sql.functions import row_number, rank, dense_rank, col, avg, lag, lead, ntile
from pyspark.sql.window import Window

df= spark.read.option("header", "True").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
#df.display()
# windowSpec = Window.orderBy(col("Item_Outlet_Sales"))
# df=df.withColumn("row_number", row_number().over(windowSpec))
# df.display()
# windowSpec = Window.orderBy(col("Item_Outlet_Sales"))
# df = df.withColumn("rank", rank().over(windowSpec))
# df.display()
# windowSpec=Window.orderBy(col("Item_Outlet_Sales"))
# df=df.withColumn("dense_rank", dense_rank().over(windowSpec))
# df.display()
# windowSpec=Window.partitionBy(col("Outlet_Location_Type")).orderBy(col("Item_Outlet_Sales"))
# df=df.withColumn("dense_rank", dense_rank().over(windowSpec))
# df.display()
# windowSpec= Window.partitionBy(col("Outlet_Location_Type"))
# df=df.withColumn("sum", sum(col("Item_Outlet_Sales")).over(windowSpec))
# df.display()
# windowSpec=Window.partitionBy(col("Outlet_Location_Type"))
# df=df.withColumn("avg", avg(col("Item_Outlet_Sales")).over(windowSpec))
# df.display()
# windowSpec=Window.orderBy(col("Item_Outlet_Sales"))
# df=df.withColumn("Prev_Sales", lag(col("Item_Outlet_Sales"), 1).over(windowSpec))
# df.display()
# windowSpec=Window.orderBy(col("Item_Outlet_Sales"))
# df=df.withColumn("Next_Sales", lead(col("Item_Outlet_Sales"),1).over(windowSpec))
# df.display()
windowSpec=Window.partitionBy("Outlet_Location_Type").orderBy(col("Item_Outlet_Sales"))
df=df.withColumn("Ntile", ntile(4).over(windowSpec))
df.display()





joins: join(),broadcast()	crossjoin(), Datetime: current_date() / current_timestamp(), datediff() / date_add() / date_sub(), to_date() / to_timestamp(), year() / month() / dayofweek(), unix_timestamp()	User Defined Functions (UDFs): udf(), pandas_udf(), sql.functions.expr()	🔹 11. Set Operations: intersect()	, union() / unionByName()	, except()	, subtract()	explode()	posexplode()	create_map() / map_keys() / map_values()	array_contains() / size()	🔹 13. Complex Types: Struct, Array, Map: withField() / dropFields()	getField()	explode()	🔹 14. File & Storage Transformations
: read.format().load()	write.format().save()	partitionBy()	repartition() / coalesce()	🔹 15. SQL-based Transformations
: spark.sql()	CREATE OR REPLACE VIEW	CTE (WITH)	MERGE INTO	🔹 16. Delta Lake-Specific Operations: MERGE, UPDATE / DELETE	OPTIMIZE / VACUUM	TIME TRAVEL	GENERATE SYNTAX FOR CHANGE DATA FEED	🔹 17. Streaming Transformations (Structured Streaming): readStream() / writeStream() trigger() watermark() outputMode("append" / "update" / "complete")
